# 丢弃法的实现

## 方法和原理

隐藏单元$h_i$$(i=1, \ldots, 5)$的计算表达式为

$$ h_i = \phi(x_1 w_1^{(i)} + x_2 w_2^{(i)} + x_3 w_3^{(i)} + x_4 w_4^{(i)} + b^{(i)}), $$

其中$\phi$是激活函数，$x_1, \ldots, x_4$是输入，$w_1^{(i)}, \ldots, w_4^{(i)}$是权重参数，$b^{(i)}$是偏差参数。设丢失概率为$p$,并设随机变量$\xi_i$有概率$p$概率为0，有$1-p$概率为1，使用丢弃法的隐藏单元$h_i$的计算表达式为：

$$ h_i = \frac{\xi_i}{1-p} \phi(x_1 w_1^{(i)} + x_2 w_2^{(i)} + x_3 w_3^{(i)} + x_4 w_4^{(i)} + b^{(i)}). $$

注意到测试模型时不使用丢弃法。由于$\mathbb{E} (\xi_i) = 1-p$，同一神经元在模型训练和测试时的输出值的期望不变。

每个神经元有可能被丢弃。$E=0 * p +1 * (1-p)=1-p$，期望是不变的。

## 实现丢弃法

根据丢弃法的定义，这里的标量`drop_probability`定义了一个`x`（`NDArray`类）中任何一个元素被丢弃的概率

In [3]:
from mxnet.gluon import nn

## 一个问号看文件，两个问号可以看实现
nn.Dense?

In [10]:
from mxnet import nd

def dropout(X, drop_probability):
    keep_probability = 1 - drop_probability
    assert 0 <= keep_probability <= 1
    # 所有的元素全部丢弃
    if keep_probability == 0:
        return X.zeros_like()
    
    # keep_probability的概率为0，1-keep_probability概率为1
    # 相乘就是1-keep_probability概率保留数据
    mask = nd.random.uniform(
        0, 1.0, X.shape, ctx=X.context) < keep_probability
    scale = 1 / keep_probability
    return mask * X * scale

运行进行验证

In [11]:
A = nd.arange(20).reshape((5,4))
dropout(A, 0.0)


[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]
 [16. 17. 18. 19.]]
<NDArray 5x4 @cpu(0)>

In [12]:
dropout(A , 0.5)


[[ 0.  2.  0.  0.]
 [ 8. 10.  0. 14.]
 [16. 18.  0. 22.]
 [ 0. 26.  0. 30.]
 [ 0.  0.  0. 38.]]
<NDArray 5x4 @cpu(0)>

In [13]:
dropout(A, 1.0)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 5x4 @cpu(0)>